<a href="https://colab.research.google.com/github/divyanshgupt/Unreliable-Transmission/blob/main/SuperSpike_offline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [52]:
#@title Dependencies
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
dtype = torch.float
device = torch.device("cpu")

# Uncomment the line below to run on GPU
#device = torch.device("cuda:0") 

### Surrogate Gradient

In [ ]:
#@title Surrogate Gradient

class SurrGradSpike(torch.autograd.Function):

  scale = 100.0 # controls the steepness of the gradient

  @staticmethod
  def forward(ctx, input):
    '''
    computes a step-function on the input. ctx is a context variable
    that stores information needed later for backpropagation
    '''
    ctx.save_for_backward(input)
    out = torch.zeros_like(input)
    out[input > 0] = 1
    return out

  @staticmethod
  def backward(ctx, grad_output):
    '''
    In the backward method, we recieve a tensor we need to compute 
    the surrogradient of the loss with respect to the input. 
    Here we use the negative half of the fast sigmoid as in 
    Zenke & Ganguli 2018.
    
    '''
    input, _ = ctx.saved_tensors
    grad_input = grad_output.clone()
    grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
    return grad

# overwrite the spike function with the surrograte gradient function
# using the apply method
spike_fn = SurrGradSpike.apply

In [ ]:
#@title Poisson Train Generator
def Poisson_trains(n, lam, timesteps, dt):
  """

  inputs:
    n - number of poisson spike trains 
    lam - 1-D array containing mean value of poisson trains
  Returns

  """
  trains = torch.zeros((n, timesteps), device=device, dtype=dtype, requires_grad=True)
  unif = torch.rand((n, timesteps), device=device, dtype=dtype, requires_grad=True)

#  counter = 0
  for i in range(n):
    trains[unif <= lam[i]*dt] = 1
#    counter += len(unif <= lam[i]*dt)
#  print("Total No. of Spikes", counter)

  return trains

### Single Neuron

In [ ]:
nb_inputs  = 100
nb_outputs = 1

nb_steps = 5000
timestep_size = 1e-4 # 0.1 msec timesteps

In [ ]:
#@title LIF Neuron Model Parameters
args = {'thres': -50,
        'U_rest': -60,
        'tau_mem': 1e-2,
        'tau_syn': 5e-3,
        'tau_ref': 5e-3,
        't_rise': 5e-3, # the pre-synaptic double exponential kernel rise time
        't_decay': 1e-2, # the pre-synaptic double exponential kernel decay time
        'timestep_size': 1e-4} 

tau_syn = args['tau_syn']
tau_mem = args['tau_mem']

alpha = float(np.exp(-timestep_size/tau_syn))
beta = float(np.exp(-timestep_size/tau_mem))
print(beta)

0.9900498337491681


In [ ]:
#@title Input Spike Trains

spk_freq = 10 # not sure about this, but assuming it since the paper
# uses 10 Hz frequency as the target output frequency (actually 
# 5 equidistant spikes over 500 ms)

input_trains = Poisson_trains(100, spk_freq*np.ones(100),
                              nb_steps, timestep_size)

RuntimeError: ignored

In [ ]:
#@title Target Spike Trains

target = torch.zeros(nb_steps)
target[:: 1000] = 1
#print(target)


In [ ]:
#@title Weight Initialization

weight_scale = 7*(1 - beta) # copied from spytorch

weights = torch.empty((nb_inputs, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(weights, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))
print("Weight Initialization Done")

Weight Initialization Done


In [ ]:
#@title SNN
def run_snn(inputs):

  h1 = torch.einsum('abc,cd->abd', (inputs,w1))
  syn = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
  mem = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
  # lists to record the membrane potentials and the synaptic currents:
  mem_rec = []
  spk_rec = []
  # loop to simulate time
  for t in range(nb_steps):
    mthr = mem - 1.0
    out = spike_fn(mthr)
    rst = out.detach()  # do not want to backpropagate through reset

    new_syn = alpha*syn + h1[:, t]
    new_mem = (beta*mem + syn)(1 - rst)

    mem_rec.append(mem)
    spk_rec.append(out)

    mem = new_mem
    syn = new_syn
  
  # create tensors to stack the elements in the recording lists
  mem_rec = torch.stack(mem_rec, dim=1)
  spk_rec = torch.stack(spk_rec, dim=1)

  # readout layer
  h2 = torch.einsum('abc,cd->abd', (spk_rec, w2))
  flt = torch.zeros((batch_size, nb_outputs), device=device, dtype=dtype)
  out = torch.zeros((batch_size, nb_outputs), device=device, dtype=dtype)
  out_rec = [out]
  for t in range(nb_steps):
    new_flt = alpha*flt + h2[:, t]
    new_out = beta*out + flt

    flt = new_flt
    out = new_out

    out_rec.append(out)

  out_rec = torch.stack(out_rec, dim=1)
  other_recs = [mem_rec, spk_rec]
  return out_rec, other_recs


In [ ]:
def run_single_neuron(input):

  syn = torch.zeros(nb_outputs, device=device, dtype=dtype)
  mem = torch.zeros(nb_outputs, device=device, dtype=dtype)

  mem_rec = []
  spk_rec = []

  for t in range(nb_steps):

    out = spike_fn(mem - 1) # since firing threshold is scaled to 1
    reset = out.detach()

    
    new_syn = alpha*syn + torch.sum(input[:, t] * weights)
    new_mem = (beta*mem + (1-beta)*new_syn)*(1 - reset)

    mem_rec.append(mem)
    spk_rec.append(out)

    mem = new_mem
    syn = new_syn
  
  mem_rec = torch.stack(mem_rec, dim=1)
  spk_rec = torch.stack(spk_rec, dim=1)

  return spk_rec, mem_rec


In [ ]:
def doubleExponential(train, args):
  z = torch.zeros(len(train), device=device, dtype=dtype)
  z_hat = torch.zeros(len(train), device=device, dtype=dtype)
  t_rise = args['t_rise']
  t_decay = args['t_decay']
  dt = timestep_size
  for t in range(len(train) - 1):
    z[t+1] =  z[t] + (-z[t]/t_rise + train[t])*dt
    z_hat[t+1] = z_hat[t] + (-z_hat[t] + z[t])*dt/t_decay

  return z_hat

In [ ]:
def van_rossum_loss(spike_train, desired_spike_train, args):

  t_rise = args['t_rise']
  t_decay = args['t_decay']

  difference = doubleExponential(desired_spike_train,
                                 args) - doubleExponential(spike_train,
                                                           args)
  loss = 1/2 * torch.sum(difference**2)*timestep_size 


  return loss

very nice description of creating custom loss functions: https://pdf.co/blog/deep-learning-pytorch-custom-loss-function


In [ ]:
class van_rossum_loss2(nn.Module):
  def __init__(self, args):
    super().__init__()
    self.t_rise = args['t_rise']
    self.t_decay = args['t_decay']
  
  def forward(self, input, target):
    

In [ ]:
params = [weights]
optimizer = torch.optim.Adam(params, lr=2e-3, betas=(0.9, 0.999))

loss_hist = []

for epochs in range(100):

  output, _ = run_single_neuron(input_trains)
  loss = van_rossum_loss(output, target, args)
 # loss = variable(van_rossum_loss, output, target, args)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  loss_hist.append(loss.item())



/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:249: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")


TypeError: ignored

In [ ]:
params = [weights] # The paramters we want to optimize
optimizer = torch.optim.Adam(params, lr=2e-3, betas=(0.9,0.999)) # The optimizer we are going to use

log_softmax_fn = nn.LogSoftmax(dim=1) # The log softmax function across output units
loss_fn = nn.NLLLoss() # The negative log likelihood loss function

# The optimization loop
loss_hist = []
for e in range(1000):
    # run the network and get output
    output,_ = run_snn(x_data) 
    # compute the loss
    m,_= torch.max(output,1)
    log_p_y = log_softmax_fn(m) 
    loss_val = loss_fn(log_p_y, y_data)

    # update the weights
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()
    
    # store loss value
    loss_hist.append(loss_val.item())
    
loss_hist_true_grad = loss_hist # store for later use